In [1]:
import logging

from automata.cli.commands import reconfigure_logging
from automata.config.base import AgentConfigName
from automata.config.openai_agent import OpenAIAutomataAgentConfigBuilder
from automata.agent.providers import OpenAIAutomataAgent
from automata.singletons.dependency_factory import dependency_factory
from automata.singletons.py_module_loader import py_module_loader
from automata.tools.factory import AgentToolFactory

logger = logging.getLogger(__name__)
reconfigure_logging("DEBUG")

py_module_loader.initialize()

In [2]:
# Construct the set of all dependencies that will be used to build the tools
toolkit_list = ["context-oracle"]
tool_dependencies = dependency_factory.build_dependencies_for_tools(toolkit_list)

INFO:automata.tools.factory:Building dependencies for toolkit_list ['context-oracle']...
INFO:automata.tools.factory:Building symbol_code_embedding_handler...
INFO:automata.tools.factory:Creating dependency symbol_code_embedding_handler
INFO:automata.tools.factory:Building symbol_search...
INFO:automata.tools.factory:Creating dependency symbol_search
INFO:automata.tools.factory:Creating dependency symbol_graph
INFO:automata.tools.factory:Creating dependency embedding_similarity_calculator
INFO:automata.tools.factory:Building embedding_similarity_calculator...
INFO:automata.tools.factory:Building symbol_doc_embedding_handler...
INFO:automata.tools.factory:Creating dependency symbol_doc_embedding_handler
INFO:automata.tools.factory:Creating dependency py_context_retriever


In [3]:
# Build the tools
tools = AgentToolFactory.build_tools(toolkit_list, **tool_dependencies)

In [4]:
# Build the agent config
config_name = AgentConfigName("automata-main")

agent_config = (
    OpenAIAutomataAgentConfigBuilder.from_name(config_name)
    .with_tools(tools)
    .with_model("gpt-4")
    .with_max_iterations(7)
    .build()
)

INFO:automata.tools.factory:Creating dependency symbol_rank
INFO:automata.tools.factory:Creating dependency subgraph
INFO:automata.symbol.graph:Pre-computing bounding boxes for all rankable symbols
Error computing bounding box for scip-python python automata 8f8ac79f556fe456d27324c5955e5c6b909ac8e1 `automata.navigation.py.dot_path_map`/DotPathMap#: Module descriptor automata.navigation.py.dot_path_map not found
Error computing bounding box for scip-python python automata 8f8ac79f556fe456d27324c5955e5c6b909ac8e1 `automata.navigation.py.dot_path_map`/DotPathMap#__init__().: Module descriptor automata.navigation.py.dot_path_map not found
Error computing bounding box for scip-python python automata 8f8ac79f556fe456d27324c5955e5c6b909ac8e1 `automata.navigation.py.dot_path_map`/DotPathMap#_build_module_dotpath_to_fpath_map().: Module descriptor automata.navigation.py.dot_path_map not found
Error computing bounding box for scip-python python automata 8f8ac79f556fe456d27324c5955e5c6b909ac8e1 `

ERROR:automata.symbol.graph:Error processing scip-python python automata 8f8ac79f556fe456d27324c5955e5c6b909ac8e1 `automata.navigation.py.dot_path_map`/DotPathMap#contains_fpath().: Symbol(scip-python python automata 8f8ac79f556fe456d27324c5955e5c6b909ac8e1 `automata.navigation.py.dot_path_map`/DotPathMap#contains_fpath()., scip-python, Package(python automata 8f8ac79f556fe456d27324c5955e5c6b909ac8e1), (Descriptor(automata.navigation.py.dot_path_map, 1), Descriptor(DotPathMap, 2), Descriptor(contains_fpath, 4)))
ERROR:automata.symbol.graph:Error processing scip-python python automata 8f8ac79f556fe456d27324c5955e5c6b909ac8e1 `automata.navigation.py.dot_path_map`/DotPathMap#get_module_dotpath_by_fpath().: Symbol(scip-python python automata 8f8ac79f556fe456d27324c5955e5c6b909ac8e1 `automata.navigation.py.dot_path_map`/DotPathMap#get_module_dotpath_by_fpath()., scip-python, Package(python automata 8f8ac79f556fe456d27324c5955e5c6b909ac8e1), (Descriptor(automata.navigation.py.dot_path_map, 1

 85%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                        | 542/634 [00:04<00:00, 133.18it/s]ERROR:automata.symbol.graph:Error processing scip-python python automata 8f8ac79f556fe456d27324c5955e5c6b909ac8e1 `automata.symbol_embedding.base`/SymbolEmbeddingHandler#get_ordered_embeddings().: Symbol(scip-python python automata 8f8ac79f556fe456d27324c5955e5c6b909ac8e1 `automata.symbol_embedding.base`/SymbolEmbeddingHandler#get_ordered_embeddings()., scip-python, Package(python automata 8f8ac79f556fe456d27324c5955e5c6b909ac8e1), (Descriptor(automata.symbol_embedding.base, 1), Descriptor(SymbolEmbeddingHandler, 2), Descriptor(get_ordered_embeddings, 4)))
ERROR:automata.symbol.graph:Error processing scip-python python automata 8f8ac79f556fe456d27324c5955e5c6b909ac8e1 `automata.symbol_embedding.base`/SymbolEmbeddingHandler#process_embedding().: Symbol(scip-python python automata 8

In [5]:
# Initialize the agent
# instructions = "Explain how OpenAI is used by the codebase. Note in particularly how it is used in embeddings and how agents consume it"
instructions = "Provide a comprehensive explanation of how SymbolRank and SymbolSearch work. Return your answer as a valid JSON string."
agent = OpenAIAutomataAgent(instructions, config=agent_config)

DEBUG:automata.agent.providers:Setting up agent with tools = [OpenAITool(function=<bound method ContextOracleToolkitBuilder._get_context of <automata.tools.builders.context_oracle.ContextOracleOpenAIToolkitBuilder object at 0x120d960d0>>, name='context-oracle', description="This tool utilizes the EmbeddingSimilarityCalculator and SymbolSearch to provide context for a given query by computing semantic similarity between the query and all available symbols' documentation and code. The symbol with the highest combined similarity score is identified, with its source code and documentation summary forming the primary context. Additionally, if enabled, the documentation summaries of related symbols (those next most similar to the query) are included.", coroutine=None, properties={'query': {'type': 'string', 'description': 'The query string to search for.'}, 'max_additional_related_symbols': {'type': 'integer', 'description': 'The maximum number of additional related symbols to return documen

DEBUG:automata.agent.providers:
------------------------------------------------------------
Session ID: ffa959b3-9798-4a27-8aff-5c077f640881
------------------------------------------------------------




In [6]:
# Run the agent
result = agent.run()

DEBUG:root:
------------------------------------------------------------------------------------------------------------------------
Latest Assistant Message -- 

DEBUG:automata.llm.providers.openai:Approximately 2415 tokens were consumed prior to completion generation.


Function Call:
context-oracle

Arguments:
{
  "query": "SymbolRank"
}

 

DEBUG:automata.llm.providers.openai:Approximately 2450 tokens were after adding the latest message.
DEBUG:root:
------------------------------------------------------------------------------------------------------------------------
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
DEBUG:automata.agent.providers:Latest User Message -- 
user:
content=Execution Result:

@dataclass
class Symbol:
    """
    A class which contains associated logic for a Symbol.

    A Symbol specifies a python class, method, or a local variable.
    A Symbol has a standardized string representation called a URI. The syntax for Symbol is the following:

    # (<x>)+ stands for one or more repetitions of <x>
    <symbol>               ::= <scheme> ' ' <package> ' ' (<descriptor>)+ | 'local ' <local-id>
    <package>              ::= <manager> ' ' <package-name> ' ' <version>
    <scheme>               ::= any UTF-8, escape spaces with double space.
    <manager>              ::= same as above, use the plac

DEBUG:automata.llm.providers.openai:Approximately 6340 tokens were after adding the latest message.
DEBUG:root:
------------------------------------------------------------------------------------------------------------------------
DEBUG:root:
------------------------------------------------------------------------------------------------------------------------
Latest Assistant Message -- 

DEBUG:automata.llm.providers.openai:Approximately 6340 tokens were consumed prior to completion generation.


Function Call:
context-oracle

Arguments:
{"query": "SymbolSearch"}

 

DEBUG:automata.llm.providers.openai:Approximately 6375 tokens were after adding the latest message.
DEBUG:root:
------------------------------------------------------------------------------------------------------------------------
DEBUG:automata.agent.providers:Latest User Message -- 
user:
content=Execution Result:

@dataclass
class Symbol:
    """
    A class which contains associated logic for a Symbol.

    A Symbol specifies a python class, method, or a local variable.
    A Symbol has a standardized string representation called a URI. The syntax for Symbol is the following:

    # (<x>)+ stands for one or more repetitions of <x>
    <symbol>               ::= <scheme> ' ' <package> ' ' (<descriptor>)+ | 'local ' <local-id>
    <package>              ::= <manager> ' ' <package-name> ' ' <version>
    <scheme>               ::= any UTF-8, escape spaces with double space.
    <manager>              ::= same as above, use the placeholder '.' to indicate an empty value
    <package-

DEBUG:automata.llm.providers.openai:Approximately 10433 tokens were after adding the latest message.
DEBUG:root:
------------------------------------------------------------------------------------------------------------------------
DEBUG:root:
------------------------------------------------------------------------------------------------------------------------
Latest Assistant Message -- 

DEBUG:automata.llm.providers.openai:Approximately 10433 tokens were consumed prior to completion generation.
INFO:openai:error_code=context_length_exceeded error_message="This model's maximum context length is 8192 tokens. However, your messages resulted in 9658 tokens (9489 in the messages, 169 in the functions). Please reduce the length of the messages or functions." error_param=messages error_type=invalid_request_error message='OpenAI API error received' stream_error=False


InvalidRequestError: This model's maximum context length is 8192 tokens. However, your messages resulted in 9658 tokens (9489 in the messages, 169 in the functions). Please reduce the length of the messages or functions.

In [ ]:
# Print the result
print(f"Result:\n{result}")

In [ ]:
result